#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (``model_name="sentence-transformers/all-mpnet-base-v2"``)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/johbaum8/DEV/build-your-own-chatbot/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/home/johbaum8/DEV/build-your-own-chatbot/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
text = "This is a test document."

query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.04895174503326416, -0.039861928671598434, -0.021562742069363594, 0.009908485226333141, -0.03810397908091545, 0.012684328481554985, 0.04349445924162865, 0.07183387875556946, 0.00974861066788435, -0.006987082771956921]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: ``PyPDFLoader``
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
loader = PyPDFLoader(pdf_doc)

# Load and split documents in chunks
pages_chunked = loader.load_and_split(text_splitter=RecursiveCharacterTextSplitter())

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

pages_chunked_cleaned = [clean_text(chunk.page_content) for chunk in pages_chunked]

print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [4]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [5]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 507


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)

In [6]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection = client.get_or_create_collection("ai_model_book")

vector_db_from_client = Chroma(
    client=client,
    collection_name="ai_model_book",
    embedding_function=embedding_model,
)

In [7]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked[:50]))]

vector_db_from_client.add_documents(documents=pages_chunked[:50], ids=uuids)

['f85b14d8-37e2-4148-891b-c46f92bb8cd6',
 '5531da75-cecd-403a-9255-137eae640c10',
 '6fa08388-0925-4899-834e-25786ffe383d',
 '6a4c6af6-e6dd-4fc3-8412-3b0b66c5e4f8',
 'fc30ddf3-763a-4c19-b5ca-6de20b1222ec',
 '2b9b2728-4048-4651-9012-8ee8faa57cc3',
 'bfe48be8-cd0c-4300-835d-733556c2186e',
 '8d6886e5-b830-4d8c-8873-3a8d4115a8dd',
 'b58c7ed6-e4cc-448f-8e7b-8f51dd2c3639',
 '45252c3e-b2b1-4250-8440-dfa5a1673553',
 'f98fdbde-c2c7-4ee9-9c92-d8b82edc2305',
 '69d1a491-8039-4cc5-a0c7-1dfe9f470ba9',
 '1559ced5-0a08-45d9-a85e-8d95b47c6fcc',
 'e8516dc2-56ed-4e3f-8b4e-03f01d945c59',
 'bf2ee221-3120-4308-a200-52dcf4d481ee',
 '74e5b8ce-a844-417a-b312-721b4ac68c8d',
 'f9f499ed-a6d2-45fc-b436-3fccfb40b656',
 '9dafac13-7bd6-4a05-b2ee-b3d26cd3efb8',
 '06ab185e-06c6-4947-971f-4d73f502e189',
 'fb1a8d6b-cf84-4c0b-978e-a8e8138877d4',
 'e4b34e4d-9cec-48e7-a0e1-e2951362e74a',
 '5d3724e8-2b38-4aea-b8df-feae9a3fa4f5',
 '723b287c-105e-497a-8e66-cdf948c512d5',
 'f34f3434-e670-4845-89ae-6d267e68947b',
 '6d8ee751-73ec-

In [8]:
client.count_collections()

1

In [89]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents


**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)

In [9]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(
    "search_query",
    k=3
)
for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Frame the Problem                                                                                                    39
Select a Performance Measure                                                                                  42
Check the Assumptions                                                                                             45
Get the Data                                                                                                                    45
Create the Workspace                                                                                                45
Download the Data                                                                                                    49
Take a Quick Look at the Data Structure                                                                50
Create a Test Set                                                                                                          54
Discover and Visualize the Data to Gain Insights